# TextExplainer

While eli5 supports many classifiers and preprocessing methods, it can't support them all. But if a library is  not supported by eli5 directly, or the text processing pipeline is too complex for eli5, eli5 can still help - it provides an implementation of LIME (Riberio et al., 2016) algorithm which allows to explain predictions of arbitrary classifiers (including text classifiers). `eli5.lime` can also help when it is hard to get exact mapping between model coefficients and text features, e.g. if there is dimension reduction involved.

## Example problem: LSA+SVM for 20 Newsgroups dataset

Let's load "20 Newsgroups" dataset and create a text processing pipeline which is hard to debug using conventional methods: SVM with RBF kernel trained on [LSA](https://en.wikipedia.org/wiki/Latent_semantic_analysis) features.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

vec = TfidfVectorizer(min_df=3, stop_words='english',
                      ngram_range=(1, 2))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(lsa, clf)
pipe.fit(twenty_train.data, twenty_train.target)
pipe.score(twenty_test.data, twenty_test.target)

0.89014647137150471

The dimension of the input documents is reduced to 100, and then a kernel SVM is used to classify the documents. 

This is what the pipeline returns for a document - it is pretty sure the first message in test data belongs to sci.med:

In [3]:
# document to explain prediction for
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(twenty_train.target_names, y_pred):
        print("{:.3f} {}".format(prob, target))    

doc = twenty_test.data[0]
print_prediction(doc)

0.000 alt.atheism
0.000 comp.graphics
0.997 sci.med
0.003 soc.religion.christian


## TextExplainer
Such pipelines are not supported by eli5 directly, but one can use eli5.lime.TextExplainer to debug the prediction - to check what was important in the document to make this decision.

Note how we pass document and predict_proba function to TextExplainer.fit method (TextExplainer works best with probabilistic classifiers):

In [4]:
from eli5.lime import TextExplainer

te = TextExplainer()
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

Weight,Feature
-0.420,<BIAS>
-10.054,Highlighted in text (sum)
Weight,Feature
-0.187,<BIAS>
-8.999,Highlighted in text (sum)
Weight,Feature
+6.861,Highlighted in text (sum)
-0.082,<BIAS>
Weight,Feature
-0.276,<BIAS>


## How does it work

Explanation makes sense, but how did we get it, and how can we be sure this is how the pipeline works? A simple sanity check is to remove or change the highlighted words, to confirm that they change the outcome:

In [5]:
import re
doc2 = re.sub(r'(recall|kidney|stones|medication|pain|tech)', '', doc)
print_prediction(doc2)

0.065 alt.atheism
0.151 comp.graphics
0.352 sci.med
0.432 soc.religion.christian


Predicted probabilities changed a lot indeed. 

And in fact, TextExplainer did something similar to get the explanation. TextExplainer generated a lot of texts similar to the document (by removing some of the words or chars), and then trained a white-box classifier which predicts the output of the black-box classifier (not the true labels!). The explanation we saw is for this white-box classifier.

This approach follows the LIME algorithm; for text data the algorithm is actually pretty straightforward:

1. generate distorted versions of the text;
2. predict probabilities for these distorted texts 
   using the black-box classifier;
3. train another classifier (one of those eli5 supports) which 
   tries to predict output of a black-box classifier on these texts.

The algorithm works because even though it could be hard or impossible to approximate a black-box classifier globally (for every possible text), approximating it in a small neighbourhood near a given text often works well, even with simple white-box classifiers.

Generated samples (distorted texts) are available in `samples_` attribute:

In [6]:
print(te.samples_[0])

   from my bout  kidney , there isn't 
medication   do    except  the .

Either  pass,         ,   
   .

   , the -  happened  mention  she'd  kidney
stones and children,  the childbirth  .


By default TextExplainer generated 5000 distorted texts (use `n_samples` argument to change the amount):

In [7]:
len(te.samples_)

5000

Trained white-box classifier and vectorizer are available as `vec_` and `clf_` attributes:

In [8]:
te.vec_, te.clf_

(CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 2), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w+\\b', tokenizer=None,
         vocabulary=None),
 SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
        learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
        penalty='elasticnet', power_t=0.5,
        random_state=<mtrand.RandomState object at 0x112c056c0>,
        shuffle=True, verbose=0, warm_start=False))

## Should we trust the explanation?

Ok, this sounds fine, but how can we be sure that this simple text classification pipeline approximated the black-box classifier well?

One way to do that is to check the quality on a held-out dataset (which is also generated). TextExplainer does that by default and stores metrics in `metrics_` attribute:

In [9]:
te.metrics_

{'mean_KL_divergence': 0.02246869327029901, 'score': 0.98189887288325461}

* 'score' is an accuracy score weighted by cosine distance between generated sample and the original document (i.e. texts which are closer to the example are more important). Accuracy shows how good are 'top 1' predictions. 
* 'mean_KL_divergence' is a mean [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) for all target classes; it is also weighted by distance. KL divergence shows how well are probabilities approximated; 0.0 means a perfect match.

In this example both accuracy and KL divergence are very good; it means our white-box classifier usually assigns the same labels as the black-box classifier on the dataset we generated, and its predicted probabilities are close to those predicted by our LSA+SVM pipeline. So it is likely (though not guaranteed, we'll discuss it later) that the explanation is correct and can be trusted.

When working with LIME (e.g. via TextExplainer) it is always a good idea to check these scores. If they are not good then you can tell that something is not right.

## Let's make it fail

By default TextExplainer uses a very basic text processing pipeline: Logistic Regression trained on bag-of-words and bag-of-bigrams features (see `te.clf_` and `te.vec_` attributes). It limits a set of black-box classifiers it can explain: because the text is seen as "bag of words/ngrams", the default white-box pipeline can't distinguish e.g. between the same word in the beginning of the document and in the end of the document. Bigrams help to alleviate the problem in practice, but not completely. 

Black-box classifiers which use features like "text length" (not directly related to tokens) can be also hard to approximate using the default bag-of-words/ngrams model. 

This kind of failure is usually detectable though - scores (accuracy and KL divergence) will be low. Let's check it on a completely synthetic example - a black-box classifier which assigns a class based on oddity of document length and on a presence of 'medication' word.

In [10]:
import numpy as np

def predict_proba_len(docs):
    # nasty predict_proba - the result is based on document length,
    # and also on a presence of "medication"
    proba = [
        [0, 0, 1.0, 0] if len(doc) % 2 or 'medication' in doc else [1.0, 0, 0, 0] 
        for doc in docs
    ]
    return np.array(proba)    

te3 = TextExplainer().fit(doc, predict_proba_len)
te3.show_prediction(target_names=twenty_train.target_names)

Weight,Feature
+6.256,Highlighted in text (sum)
-0.011,<BIAS>


TextExplainer correctly figured out that 'medication' is important, but failed to account for "len(doc) % 2" condition, so the explanation is incomplete. We can detect this failure by looking at metrics - they are low:

In [11]:
te3.metrics_

{'mean_KL_divergence': 0.30581890571484338, 'score': 0.79670704047322449}

If (a big if...) we suspect that the fact document length is even or odd is important, it is possible to customize TextExplainer to check this hypothesis. 

To do that, we need to create a vectorizer which returns both "is odd" feature and bag-of-words features, and pass this vectorizer to TextExplainer. This vectorizer should follow scikit-learn API. The easiest way is to use FeatureUnion - just make sure all transformers joined by FeatureUnion have get_feature_names() methods.

In [12]:
from sklearn.pipeline import make_union
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin

class DocLength(TransformerMixin):
    def fit(self, X, y=None):  # some boilerplate
        return self
    
    def transform(self, X):
        return [
            # note that we needed both positive and negative 
            # feature - otherwise for linear model there won't 
            # be a feature to show in 50% cases
            [len(doc) % 2, not len(doc) % 2] 
            for doc in X
        ]
    
    def get_feature_names(self):
        return ['is_odd', 'is_even']

vec = make_union(DocLength(), CountVectorizer(ngram_range=(1,2)))
te4 = TextExplainer(vec=vec).fit(doc[:-1], predict_proba_len)

print(te4.metrics_)
te4.explain_prediction(target_names=twenty_train.target_names)

{'score': 1.0, 'mean_KL_divergence': 0.024643780992576535}


Weight,Feature
+8.684,countvectorizer: Highlighted in text (sum)
-0.064,<BIAS>
-3.127,doclength__is_even


Much better! It was a toy example, but the idea stands - if you think something could be important, add it to the mix as a feature for TextExplainer.

## Let's make it fail, again

Another possible issue is the dataset generation method. Not only feature extraction should be powerful enough, but auto-generated texts also should be diverse enough. 

TextExplainer removes random words by default, so by default it can't e.g. provide a good explanation for a black-box classifier which works on character level. To explain such black-box classifiers we need to change both dataset generation method (change/remove individual characters, not only words) and feature extraction method (e.g. use char ngrams instead of words and word ngrams). 

If this makes a more powerful explanation engine why not always use it? The problem is that it is much more resource intensive - you need a lot more samples to get non-noisy results. Generally speaking, to do an efficient explanation we should make some assumptions about black-box classifier, such as:

1. it uses words as features and doesn't take word position in account;
2. it uses words as features and takes word positions in account;
3. it uses words ngrams as features (it is partially covered by (2));
4. it uses char ngrams as features, positions don't matter (i.e. an ngram means the same everywhere);
5. it uses arbitrary attention over the text characters, i.e. every part of text could be potentionally important for a classifier on its own;
6. it is important to have a particular token at a particular position, e.g. "third token is X", and if we delete 2nd token then prediction changes not because 2nd token changed, but because 3rd token is shifted.

Depending on assumptions we should choose both dataset generation method and a white-box classifier. There is a tradeoff between genereality and speed. Simple bag-of-words assumptions allow for fast sample generation, and just a few hundreds of samples could be required to get an OK quality if the assumption is correct. But such generation methods / models will fail to explain a more complex classifier properly. On the other hand, allowing for each character to be important is a more powerful method, but it can require a lot of samples (maybe hundreds thousands) and a lot of CPU time to get non-noisy results.

What's bad about this kind of failure (wrong assumption about the black-box pipeline) is that it could be impossible to detect the failure by looking at the scores. Scores could be high because generated dataset is not diverse enough, not because our approximation is good.

It means that it is important to understand the "lenses" you're looking through when using LIME to explain a prediction.